In [1]:
    if __name__ == '__main__':
        %reload_ext importable
    from contextlib import contextmanager
    from fnmatch import fnmatch
    from functools import partial
    from importable import finder, unload
    from IPython import get_ipython, display
    from IPython.core.interactiveshell import InteractiveShell
    from mistune import Markdown, Renderer
    from nbformat.v4 import new_notebook, new_code_cell, new_markdown_cell
    from nbconvert import export, get_exporter
    from nbconvert.exporters.python import PythonExporter    
    from toolz.curried import compose, do, identity
    import traitlets
    from types import MethodType
    from mimetypes import MimeTypes; mimetypes = MimeTypes()

In [2]:
    def ipython2python(code):
        from nbconvert.filters.strings import ipython2python
        if any(map(code.startswith, '!%')):
            code = ipython2python(code)
        return code

In [28]:
    def macro(code):
        from IPython import display
        type = mimetypes.guess_type(code)[0]
        is_image = type and type.startswith('image')
        disp = (
            partial(display.Image, embed=True) 
            if is_image else display.Markdown)
        if fnmatch(code, '*[[]*[]](*)'):
            url = (
                code.lstrip('#').lstrip()
                .split(']', 1)[1].lstrip('(')
                .rstrip(')').split(' ',1)[0])
            if url and url != '#':
                return (display.Markdown(code), *macro(url))
        if fnmatch(code, 'http*://*'):
            if is_image: 
                return display.Image(url=code),
            return display.IFrame(code, width=600, height=400),
        try:
            if __import__('pathlib').Path(code).is_file(): 
                return disp(filename=code),
        except OSError:
            pass

In [15]:
    class LiterateShell(InteractiveShell):
        weave = traitlets.Any(default_value=identity)
        tangle = traitlets.Any(default_value=identity)
        macro = traitlets.Any(default_value=macro)
        name = traitlets.Unicode('literate')
        parent = traitlets.Any(get_ipython())
        
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            
            self.parent and self.parent.register_magic_function(
                self, magic_kind='line_cell', magic_name=self.name)
            
        def __call__(self, line, code=""""""):
            code = '\n'.join(filter(bool, [line, code])).rstrip()
            code and self.run_cell(code)
            
        def preprocess(self, text, output=None, show=True):
            text = self.weave(text)
            
            if show and text.splitlines() and text.splitlines()[0].strip():
                if len(text.splitlines()) is 1:
                    output = self.macro(text)
                display.display(*(output or [display.Markdown(text)]))                
            
            code = self.tangle(text)
            if any(map(code.startswith, '!%')): 
                code = ipython2python(code)
            return code
            
        def run_cell(self, text, store_history=False, silent=False, shell_features=True):
            return InteractiveShell.run_cell(self.parent, self.preprocess(text), store_history, silent, shell_features)

In [16]:
    class Weave(Renderer):
        """A mistune.Renderer to accumulate lines of code in a Markdown document."""
        code = """"""
        inline_code = True
        inline_indent = True
        
        def block_code(self, code, lang=None):
            self.code += code + '\n'
            return super(Weave, self).block_code(code, lang)

        def codespan(self, code):
            """Weave inline code references
            """
            if self.inline_code:
                if self.inline_indent:
                    self.code += ' '*self.indents(self.code) + code + '\n'
                else:
                    self.code += code + '\n'
            return super(Weave, self).codespan(code)
        
        @staticmethod
        def indents(code):
            """Determine the indent length of the last line."""
            if code:
                str = list(filter(lambda s: s.strip(), code.splitlines()))
                if str:  
                    return len(str[-1])-len(str[-1].lstrip())
            return 0


    class Tangle(Markdown):
        """A mistune.Markdown processor for literate programming."""
        def render(self, text, **kwargs):
            """Render ingests Markdown tests and returns executable portions"""
            text=self.rstrip(text)
            self.renderer.code = """"""
            super(Tangle, self).render(text, **kwargs)
            data =  '\n'.join(str.rstrip('    '+s) for s in self.renderer.code.splitlines())
            if not data.strip(): return """"""
            code = PythonExporter().from_notebook_node(
                new_notebook(cells=[new_code_cell(data)]))[0]
            return code

        @staticmethod
        def rstrip(text):
            """Remove whitespace for diffing later."""
            return '\n'.join(map(str.rstrip, map(str, text.splitlines())))

In [17]:
    literate = Tangle(renderer=Weave(), escape=False)

In [18]:
    class Execute(__import__('nbconvert').preprocessors.execute.ExecutePreprocessor):
        tangle = staticmethod(literate.render)
        def run_cell(self, code, index=0):
            code.source = ipython2python(self.tangle(code.source))
            return super().run_cell(code, index)

In [22]:
    def extension(shell):
        def load_ipython_extension(ip=get_ipython()):
            ip.run_cell = shell.run_cell
            loader(partial(shell.preprocess, show=False))
                
        def unload_ipython_extension(ip=get_ipython()):
            unload(['ipynb'])
            ip.run_cell = InteractiveShell.run_cell
        
        return load_ipython_extension, unload_ipython_extension

In [23]:
    exporter = get_exporter('script')

    def nb_to_python(callable, nb):
        new = []
        for cell in nb.cells:
            if cell['cell_type'] == 'code':
                cell.source = callable(
                    '\n'.join(cell.source) if isinstance(cell.source, list) else cell.source)
                new.append(cell)
            nb.cells = new
        return export(exporter, nb)[0].encode('utf-8')

    def loader(render):
        def wrapped(self, path):
            nonlocal render
            with open(self.path) as f:
                out = nb_to_python(render, __import__('nbformat').read(f, 4))
            return out
        return unload(['ipynb']) or finder('ipynb')(wrapped)

In [24]:
    load_ipython_extension, unload_ipython_extension = extension(
        LiterateShell(
            tangle=literate.render, macro=macro, parent=get_ipython()
        ))
    load = load_ipython_extension